# Quant Volatility Surface & Pricing Engine Demo

This notebook demonstrates an end-to-end quantitative finance pipeline for:

1.  **Data ETL**: Fetching real-time SPY options data.
2.  **SVI Calibration**: Fitting the Stochastic Volatility Inspired model to market smiles.
3.  **Local Volatility**: Extracting Dupire's Local Volatility surface.
4.  **Exotic Pricing**: Pricing a **Down-and-Out Call Option** using Monte Carlo simulations.
5.  **Hedging Analysis**: Comparing Black-Scholes Delta vs. Local Vol Delta.

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Ensure 'src' is in the python path
if 'src' not in sys.path:
    sys.path.append(os.path.abspath(''))

# Import core project modules
from src.vol_surface import VolatilitySurface
from src.pricer import MonteCarloPricer
from src.rates import RateProvider

print(" Environment setup complete. Project modules loaded.")

 Environment setup complete. Project modules loaded.


## 1. Data ETL & Surface Construction

We initialize the `VolatilitySurface` object. This triggers:
* Fetching live option chain for **SPY** from Yahoo Finance.
* Cleaning data (removing zero-volume/zero-vol quotes).
* Calibrating the **Raw SVI parameters** ($a, b, \rho, m, \sigma$) for every expiration slice.

In [2]:

TICKER = "SPY"
print(f" Initializing Volatility Surface for {TICKER}...")

# Initialize the engine
rates = RateProvider()
surface = VolatilitySurface(TICKER)

# Run the build pipeline (Fetch -> Clean -> Calibrate)
surface.build()

q = surface.data_loader.fetch_dividend_yield()

print(f"\n Surface Built Successfully!")
if surface.spot_price:
    print(f" Spot Price: ${surface.spot_price:.2f}")
print(f" Calibrated Expirations: {len(surface.svi_params)} slices")

# Show a sample of the cleaned raw data
print("\n Sample Raw Data (Top 5 rows):")
display(surface.raw_data.head())

2025-12-31 12:39:35,273 - INFO - Fetching Treasury Yield Curve data from Yahoo Finance...


 Initializing Volatility Surface for SPY...


2025-12-31 12:39:37,396 - INFO - Yield Curve constructed using 4 points. 10Y Rate: 4.13%
2025-12-31 12:39:37,399 - INFO - Fetching Treasury Yield Curve data from Yahoo Finance...
2025-12-31 12:39:37,647 - INFO - Yield Curve constructed using 4 points. 10Y Rate: 4.13%
2025-12-31 12:39:38,210 - INFO - Current Spot Price for SPY: 687.01
2025-12-31 12:39:38,211 - INFO - Dividend Yield (q) fetched: 1.0600%
2025-12-31 12:39:38,602 - INFO - Found 28 expiration dates. Starting download...
2025-12-31 12:39:46,580 - INFO - Raw data fetched: 7499 rows.
2025-12-31 12:39:46,596 - INFO - Cleaning complete. Final dataset: 3909 rows.
2025-12-31 12:39:46,598 - INFO - Dividend Yield (q) fetched: 1.0600%
2025-12-31 12:39:46,600 - INFO - Building Vol Surface for SPY (Log-Forward) across 27 expiries.
2025-12-31 12:39:46,701 - WARNING - Potential Calendar Arbitrage at T=0.0123. Variance decreased. Skipping.
2025-12-31 12:39:46,895 - WARNING - Potential Calendar Arbitrage at T=0.0205. Variance decreased. Ski


 Surface Built Successfully!
 Spot Price: $687.01
 Calibrated Expirations: 21 slices

 Sample Raw Data (Top 5 rows):


,contractSymbol,expirationDate,T,K,moneyness,optionType,bid,ask,impliedVolatility,volume,openInterest
552,SPY260102C00688000,2026-01-02,0.004034,688.0,1.001441,call,1.86,1.89,0.094614,40390.0,7007.0
553,SPY260102C00689000,2026-01-02,0.004034,689.0,1.002897,call,1.40,1.42,0.091562,16127.0,4396.0
554,SPY260102C00690000,2026-01-02,0.004034,690.0,1.004352,call,1.01,1.03,0.088754,32892.0,36907.0
555,SPY260102C00691000,2026-01-02,0.004034,691.0,1.005808,call,0.70,0.71,0.085825,11489.0,9682.0
556,SPY260102C00692000,2026-01-02,0.004034,692.0,1.007263,call,0.47,0.48,0.083994,12292.0,6613.0


## 2. SVI Calibration Inspection (2D Smile)

Let's inspect a specific maturity slice to see how well the SVI model fits the raw market data.
We look for the characteristic "Smile" or "Skew" shape typical of Equity markets (Left high, Right low).

In [3]:
# Pick a middle expiration (e.g., closest to 0.5 years or 1 year)
sorted_expirations = sorted(surface.svi_params.keys())
if not sorted_expirations:
    print("No expirations found. Check connection.")
else:
    target_T = sorted_expirations[len(sorted_expirations)//2] # Pick median expiry

    print(f" Inspecting Smile for T = {target_T:.4f} Years")

    # 1. Get Market Data points
    slice_data = surface.raw_data[surface.raw_data['T'] == target_T]
    k_market = np.log(slice_data['moneyness'])
    vol_market = slice_data['impliedVolatility']

    # 2. Generate Model Curve
    k_grid = np.linspace(k_market.min()-0.1, k_market.max()+0.1, 100)
    vol_model = [surface.get_implied_vol(k, target_T) for k in k_grid]
    m_grid = np.exp(k_grid) # Convert log-moneyness back to Strike/Spot ratio

    # 3. Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=slice_data['moneyness'], y=vol_market, mode='markers', name='Market Data', marker=dict(color='red', size=8, symbol='x')))
    fig.add_trace(go.Scatter(x=m_grid, y=vol_model, mode='lines', name='SVI Model', line=dict(color='blue', width=3)))

    fig.update_layout(
        title=f"SVI Calibration: {TICKER} (T={target_T:.2f} Years)",
        xaxis_title="Moneyness (Strike / Spot)",
        yaxis_title="Implied Volatility",
        template="plotly_white",
        height=500
    )
    fig.show()

 Inspecting Smile for T = 0.3273 Years


## 3. Implied Volatility Surface (3D)

We interpolate the Total Variance ($w = \sigma^2 T$) in the time dimension to create a dense grid.
This represents the **Market's Average Expectation** of volatility over the life of the option.

In [4]:
# Get mesh grid coordinates
X, Y, Z_imp, Z_loc = surface.get_mesh_grid()

fig_imp = go.Figure(data=[go.Surface(z=Z_imp, x=X, y=Y, colorscale='Viridis')])

fig_imp.update_layout(
    title=f'{TICKER} Implied Volatility Surface',
    scene=dict(
        xaxis_title='Moneyness (K/S)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Implied Vol'
    ),
    width=900, height=600,
    margin=dict(l=65, r=50, b=65, t=90)
)
fig_imp.show()

## 4. Local Volatility Surface (Dupire)

Using **Dupire's Formula**, we extract the *instantaneous* volatility $\sigma_{loc}(S, t)$.

Notice how the Local Volatility surface is much **steeper and spikier** than the Implied Volatility surface. 
It amplifies the skew to mathematically explain the market prices for path-dependent pricing.

In [5]:
fig_loc = go.Figure(data=[go.Surface(z=Z_loc, x=X, y=Y, colorscale='Turbo')])

fig_loc.update_layout(
    title=f'{TICKER} Local Volatility Surface (Dupire)',
    scene=dict(
        xaxis_title='Moneyness (Spot/Strike)',
        yaxis_title='Time to Maturity (Years)',
        zaxis_title='Local Vol'
    ),
    width=900, height=600,
    margin=dict(l=65, r=50, b=65, t=90)
)
fig_loc.show()

## 5. Exotic Pricing: Down-and-Out Call

We price a **Down-and-Out Call Option**.
* **Structure:** If the asset price drops below the Barrier, the option becomes worthless.
* **Model Comparison:**
    * **Black-Scholes:** Assumes constant volatility (ATM).
    * **Local Volatility:** Uses the dynamic surface derived above.

**Hypothesis:** Local Volatility should result in a lower price (or different risk profile) because it captures the **Skew** (higher volatility on the downside increases knock-out probability).

In [6]:
# Parameters
if surface.spot_price:
    S0 = surface.spot_price
    K = float(int(S0 * 1.05))       # Strike 5% OTM
    Barrier = float(int(S0 * 0.85)) # Barrier 15% below Spot
    T = 1.0             # 1 Year maturity
    N_SIMS = 50000       # Monte Carlo paths

    print(f" Pricing Parameters:")
    print(f"   Spot: ${S0:.2f} | Strike: ${K:.2f} | Barrier: ${Barrier:.2f}")
    print(f"   Dividend Yield: {q:.4%}")

    # Initialize Pricer
    pricer = MonteCarloPricer(S0, T, rates, q, surface)
    
    # 1. Black-Scholes Price (Benchmark)
    atm_vol = surface.get_implied_vol(0, T) # k=0 is ATM
    res_bs = pricer.price_barrier_option(K, Barrier, model="black_scholes", const_vol=atm_vol, n_paths=N_SIMS)

    # 2. Local Vol Price
    res_lv = pricer.price_barrier_option(K, Barrier, model="local_vol", n_paths=N_SIMS)

    # Output
    print("\n Pricing Results:")
    print("-" * 40)
    print(f"ATM Volatility Used:   {atm_vol:.2%}")
    print(f"Black-Scholes Price:   ${res_bs['price']:.2f}")
    print(f"Local Vol Price:       ${res_lv['price']:.2f}")
    print("-" * 40)
    print(f"Difference (Model Risk): ${res_lv['price'] - res_bs['price']:.2f}")
else:
    print("Error: Spot price not available.")

 Pricing Parameters:
   Spot: $687.01 | Strike: $721.00 | Barrier: $583.00
   Dividend Yield: 1.0600%


2025-12-31 12:39:49,002 - INFO - Pre-computing Local Volatility Grid...



 Pricing Results:
----------------------------------------
ATM Volatility Used:   17.34%
Black-Scholes Price:   $39.85
Local Vol Price:       $38.53
----------------------------------------
Difference (Model Risk): $-1.32


## 6. Hedging Analysis: Delta Profile

We calculate the **Delta (Hedge Ratio)** across a range of spot prices as the price approaches the Barrier.

This visualization reveals **Model Risk**:
* **Black-Scholes Delta (Gray):** Often underestimates the risk of the barrier.
* **Local Vol Delta (Red):** Adapts to the increasing volatility as spot drops, suggesting a different hedging strategy.

In [7]:
print(" Calculating Delta Profile ")

if surface.spot_price:
    # Generate Spot Range (From Barrier to ITM)
    spot_range = np.linspace(Barrier * 0.95, S0 * 1.1, 15)
    bs_deltas = []
    lv_deltas = []
    
    # Progress update
    print(f"Simulating Delta across {len(spot_range)} spot levels with {N_SIMS} paths...")

    for s_val in spot_range:
        # Temp pricer for hypothetical spot
        temp_pricer = MonteCarloPricer(s_val, T, rates, q, surface)        
        
        # Calculate Deltas 
        d_bs = temp_pricer.calculate_delta(K, Barrier, model="black_scholes", n_paths=N_SIMS)
        d_lv = temp_pricer.calculate_delta(K, Barrier, model="local_vol", n_paths=N_SIMS)
        
        bs_deltas.append(d_bs)
        lv_deltas.append(d_lv)

    # Plot
    fig_delta = go.Figure()
    fig_delta.add_trace(go.Scatter(x=spot_range, y=bs_deltas, mode='lines+markers', name='BS Delta', line=dict(color='gray', dash='dash')))
    fig_delta.add_trace(go.Scatter(x=spot_range, y=lv_deltas, mode='lines+markers', name='Local Vol Delta', line=dict(color='red', width=3)))
    fig_delta.add_vline(x=Barrier, line_width=2, line_dash="dot", line_color="black", annotation_text="Barrier")

    fig_delta.update_layout(
        title=f"Delta Skew: Hedging Ratio vs. Spot Price (N_SIMS={N_SIMS})",
        xaxis_title="Spot Price",
        yaxis_title="Option Delta",
        template="plotly_white"
    )
    fig_delta.show()

 Calculating Delta Profile 
Simulating Delta across 15 spot levels with 50000 paths...
